In [1]:
import numpy as np
import scipy.sparse as sp
from mpl_toolkits import mplot3d
from rom_am import DMDc, ROM, DMD
import matplotlib.pyplot as plt

## Unstable Linear system

We consider the following unstable linear system:

$$
\begin{pmatrix}
x_1  \\
x_2 \\
\end{pmatrix}_{k+1} = \begin{pmatrix}
1.5 & 0  \\
0 & 0.1 \\
\end{pmatrix} \begin{pmatrix}
x_1  \\
x_2 \\
\end{pmatrix}_{k} + \begin{pmatrix}
-3  \\
0 \\
\end{pmatrix} u_k $$

Which should be modeled correctly by the DMD with control (DMDc), (of which the Affine DMD is a special case, when the input is fixed) whereas it can't be using the classical DMD.

We will use the DMDc to learn $\pmb{b} = \begin{pmatrix}
-3  \\
0 \\
\end{pmatrix}$ as well as the system matrix $\pmb{A}$

**Creating the snapshots**

In [2]:
X = np.zeros((2, 4))
Y = np.zeros((2, 4))
X[:, 0] = np.array([4, 7])

A = np.array([[1.5, 0], [0, 0.1]])
B = np.array([-3, 0]).reshape((-1, 1))
u = np.ones(4)

Y[:, 0] = A @ X[:, 0] + B.ravel() * u[0]
for i in range(Y.shape[1]-1):
    Y[:, i+1] = A @ Y[:, i] + B.ravel() * u[i]
X[:, 1::] = Y[:, :-1]

**DMDc Learning**

We train the DMDc model on the extracted snapshots

In [3]:
#u_input = np.ones((1, X.shape[1]))
u_input = u.reshape((1, -1))
dt = 0.1 
rank = 0
dmdc = DMDc()
dromc = ROM(dmdc)
dromc.decompose(X,  Y = Y, dt = dt, rank =rank, Y_input=u_input)

We can show now the learned matrix $\pmb{A}$

In [4]:
dromc.model.u_hat @ dromc.model.A_tilde @ dromc.model.u_hat.T

array([[ 1.50000000e+00, -6.66132691e-16],
       [-4.92613273e-17,  1.00000000e-01]])

and the learned matrix $\pmb{b}$

In [5]:
dromc.model.u_hat @ dromc.model.B_tilde

array([[-3.00000000e+00],
       [ 5.55111512e-17]])

-------------------------------------------------------------------------------------------------------------------------------------------------

**Question : Can DMD learn it too ?**

Obviously, classical DMD is not able to learn a $\pmb{X}^{k+1} = \pmb{A} \pmb{X}^{k} + \pmb{b}$. And it approximates it through a $\pmb{X}^{k+1} = \pmb{A} \pmb{X}^{k}$ model

In [6]:
u_input = u.reshape((1, -1))
dt = 0.1 
rank = 0
dmd_ = DMD()
drom_ = ROM(dmd_)
drom_.decompose(X,  Y = Y, dt = dt, rank =rank,)

$\pmb{A}$ = 

In [7]:
drom_.modes @ drom_.model.A_tilde @ drom_.modes.T

array([[5.39578170e-01, 1.14627842e-01],
       [1.11900706e-16, 1.00000000e-01]])

In [8]:
error_dmd = np.linalg.norm(Y - drom_.modes @ drom_.model.A_tilde @ drom_.modes.T @ X)
error_dmdc = np.linalg.norm(Y - (dromc.model.u_hat @ dromc.model.A_tilde @ dromc.model.u_hat.T @ X + dromc.model.u_hat @ dromc.model.B_tilde @ u_input))

In [9]:
print("DMD error : "+ str(error_dmd)+"\nDMDc error : "+str(error_dmdc))

DMD error : 4.04284239297286
DMDc error : 2.3337360971660245e-15


------------------------------------------------------------------------------------------------------------------------

Answer : In this case, **No**